In [1]:
# Install required packages using pip
%pip install pypdf langchain_community langchain langchain_openai neo4j_genai langchain_experimental IPython neo4j yfiles_jupyter_graphs yfiles_jupyter_graphs_for_neo4j==1.3.1


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Initialization
This section initializes the notebook by importing necessary libraries and loading environment variables.

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
import ast

# Check if running in Google Colab and enable custom widget manager if true
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [3]:
# Load environment variables from 'credentials.env' if it exists
if os.path.exists('credentials.env'):
    load_dotenv('credentials.env', override=True)

    # Neo4j credentials
    uri = os.getenv('NEO4J_URI')
    username = os.getenv('NEO4J_USERNAME')
    password = os.getenv('NEO4J_PASSWORD')
    database = os.getenv('NEO4J_DATABASE')

    # OpenAI credentials
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
else:
    print("File 'credentials.env' not found.")

## Create Text Chunks and Generate Embeddings
This section splits the PDF document into chunks and generates embeddings for each chunk using OpenAI.

In [4]:
# Initialize text splitter with specified chunk size and overlap
chunk_size = 800
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
    is_separator_regex = False,
)

In [5]:
# Load and split the PDF document into chunks
doc_path = 'microsoft-blogpost.pdf'
all_chunks = []

loader = PyPDFLoader(doc_path)
pages = loader.load_and_split()
for page in pages:
    chunks = text_splitter.split_text(page.page_content)
    for chunk in chunks:
        all_chunks.append(chunk)
        
print(f"Parsed: {doc_path}")
print(f"Chunked {len(pages)} pages into {len(all_chunks)} chunks")

Parsed: microsoft-blogpost.pdf
Chunked 8 pages into 25 chunks


In [6]:
# Check the first two chunks to ensure correct splitting
print('\nFirst chunk: \n' + all_chunks[0])
print('\nSecond chunk: \n' + all_chunks[1])


First chunk: 
The Microsoft Ecosystem and Its Advantages
Alpesh Patel·Follow
9 min read·May 22, 2023
Listen Share
Digital business transformation, in today’s hi-tech marketplace has become a
business reality. For our businesses to create value for customers, it is imperative
that our evolving business and rapidly changing technology landscape adopts
transformation to create value for the customer. Therefore, every business today
must transform to address new competitive forces and create new value. Digital
business transformation, if done strategically, brings a deep understanding of
customer, thereby, enabling you to design product and services that best meet their
needs, streamline operations through application that improves responsiveness,

Second chunk: 
needs, streamline operations through application that improves responsiveness,
service levels, and reduce costs, and delivers more empowered teams by providing
them the right information at the right time.
There are 4 pillars of 

In [7]:
# Generate embeddings for each chunk using OpenAI model
model = 'text-embedding-3-small'
embeddings_model = OpenAIEmbeddings(
    model = model,
    openai_api_key = OPENAI_API_KEY
)
embeddings = []
for chunk in all_chunks:
    embeddings.append(embeddings_model.embed_query(chunk))

In [8]:
# Check the first embedding to ensure correct generation
print(all_chunks[0])
print('\nFirst embedding (sample): \n' + str(embeddings[0][1:100]))

The Microsoft Ecosystem and Its Advantages
Alpesh Patel·Follow
9 min read·May 22, 2023
Listen Share
Digital business transformation, in today’s hi-tech marketplace has become a
business reality. For our businesses to create value for customers, it is imperative
that our evolving business and rapidly changing technology landscape adopts
transformation to create value for the customer. Therefore, every business today
must transform to address new competitive forces and create new value. Digital
business transformation, if done strategically, brings a deep understanding of
customer, thereby, enabling you to design product and services that best meet their
needs, streamline operations through application that improves responsiveness,

First embedding (sample): 
[-0.007180939894169569, 0.055880360305309296, 0.06044750660657883, 0.05364156514406204, -0.04005207121372223, 0.012850691564381123, 0.08686530590057373, -0.015783514827489853, 0.013231287710368633, 0.04728338122367859, -0.0216267742

## Write Data to Neo4j
This section shows how to clear the existing database, create document nodes, and link text chunks with their embeddings in Neo4j.

In [9]:
# Connect to Neo4j and clear the database
driver = GraphDatabase.driver(uri, auth=(username, password), database=database)
driver.execute_query('MATCH (n) DETACH DELETE n')
print("Database cleared.")

Database cleared.


In [10]:
driver.execute_query("""
CREATE VECTOR INDEX vecindex IF NOT EXISTS
FOR (m:Chunk)
ON m.embedding
OPTIONS {indexConfig: {
 `vector.dimensions`: 1536,
 `vector.similarity_function`: 'cosine'
}}""")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x127f52610>, keys=[])

In [11]:
# Create a document node for the PDF
driver.execute_query('CREATE (d:Document{name:"'+doc_path+'"})')
print('Document created.')

Document created.


In [12]:
# Create chunk nodes and link them to the document node
for index, chunk in enumerate(all_chunks):
    embedding = embeddings[index]
    driver.execute_query(f"""
    MATCH (d:Document)
    WHERE d.name = '{doc_path}'
    CREATE (d)-[:HAS_CHUNK]->(c:Chunk)
    SET c.text = '{chunk}'
    SET c.embedding = {embedding}
    SET c.index = {index}
    """)
    
print('Embeddings set.')

Embeddings set.


## Visualize the Graph
Use yFiles Jupyter graphs for Neo4j to visualize the document and its chunks.

In [13]:
# Visualize the document and chunks in Neo4j
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
widget = Neo4jGraphWidget(driver, overview_enabled=False, context_start_with=None)
widget.show_cypher("MATCH (d:Document)-[r]->(c:Chunk) RETURN d,r,c")

GraphWidget(layout=Layout(height='760px', width='100%'))

## Extract Graph from Text
Use LangChain to extract entities from the graph.

In [14]:
### Graph Schema
# either one can be set to 'None' to let the model infer data.
allowed_nodes = ["Capability", "Service", "Organization", "Department", "Industry", "DataSource", "Person", "Article"]
allowed_relationships = ["PROVIDED_BY","HAS_SERVICE","ENABLES_CAPABILITY","USED_BY","USES_DATA_FROM","BELONGS_TO_DEPARTMENT","CONNECTS_TO"]


In [15]:
# Import necessary libraries and initialize LangChain's LLMGraphTransformer
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
llm_transformer = LLMGraphTransformer(
    llm=llm, 
    allowed_nodes=allowed_nodes, 
    allowed_relationships=allowed_relationships)
from langchain_core.documents import Document

# Initialize dictionaries to store nodes and relationships
nodes = {}
rels = {}

# Define functions to generate unique hashes for nodes and relationships
def get_node_hash(node):
    return hash(node.id + ':' + node.type)

def get_rel_hash(rel):
    return hash(rel.source.id + ':' + rel.source.type + ':' + rel.type + ':' + rel.target.id + ':' + rel.target.type)

# Process each text chunk to extract graph information
for index, chunk in enumerate(all_chunks):
    documents = [Document(page_content=chunk)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)

    # Extract unique nodes from the LangChain output
    for node in graph_documents[0].nodes:
        node_hash = get_node_hash(node)
        if node_hash in nodes:
            nodes[node_hash]['chunks'].append(index)
        else:
            nodes[node_hash] = {'id': node_hash, 'name': node.id, 'label': node.type, 'chunks': [index]}

    # Extract unique relationships from the LangChain output
    for rel in graph_documents[0].relationships:
        rel_hash = get_rel_hash(rel)
        if rel_hash in rels:
            rels[rel_hash]['chunks'].append(index)
        else:
            source_hash = get_node_hash(rel.source)
            target_hash = get_node_hash(rel.target)
            rels[rel_hash] = {'id': rel_hash, 'source': source_hash, 'target': target_hash, 'type': rel.type, 'chunks': [index]}

    print(f"Loaded chunk {index+1}/{len(all_chunks)} Current nodes: {len(nodes)}, relationships: {len(rels)}...")

print('Complete')

Loaded chunk 1/25 Current nodes: 3, relationships: 2...
Loaded chunk 2/25 Current nodes: 12, relationships: 10...
Loaded chunk 3/25 Current nodes: 29, relationships: 26...
Loaded chunk 4/25 Current nodes: 41, relationships: 37...
Loaded chunk 5/25 Current nodes: 51, relationships: 43...
Loaded chunk 6/25 Current nodes: 51, relationships: 43...
Loaded chunk 7/25 Current nodes: 54, relationships: 43...
Loaded chunk 8/25 Current nodes: 59, relationships: 43...
Loaded chunk 9/25 Current nodes: 71, relationships: 55...
Loaded chunk 10/25 Current nodes: 71, relationships: 57...
Loaded chunk 11/25 Current nodes: 78, relationships: 62...
Loaded chunk 12/25 Current nodes: 79, relationships: 62...
Loaded chunk 13/25 Current nodes: 87, relationships: 70...
Loaded chunk 14/25 Current nodes: 88, relationships: 74...
Loaded chunk 15/25 Current nodes: 102, relationships: 89...
Loaded chunk 16/25 Current nodes: 113, relationships: 100...
Loaded chunk 17/25 Current nodes: 119, relationships: 105...
Loa

In [16]:
# Write nodes to Neo4j and link them to the corresponding chunks
## TODO -> this needs some optimization using parameters / batching.
for node in nodes.values():
    driver.execute_query(f"""
    CREATE (n:{node['label']})
    SET n.name = "{node['name']}"
    SET n.id = "{node['id']}"
    WITH n
    UNWIND {str(node['chunks'])} as chunk_index
    MATCH (c:Chunk)
    WHERE c.index = chunk_index
    CREATE (n)-[:IN]->(c)
    """)

print('Nodes created and linked to chunks.')

Nodes created and linked to chunks.


In [17]:
# Write relationships to Neo4j
## TODO -> this needs some optimization using parameters / batching.
for rel in rels.values():
    driver.execute_query(f"""
    MATCH (n), (m)
    WHERE n.id = "{rel['source']}" AND m.id = "{rel['target']}"
    CREATE (n)-[r:{rel['type']}]->(m)
    SET r.chunks = {str(rel['chunks'])}
    SET r.id = {str(rel['id'])}
    """)

print('Relationships created.')

Relationships created.


In [18]:
# Visualize the graph in Neo4j
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
widget = Neo4jGraphWidget(driver, overview_enabled=False, context_start_with=None)
widget.show_cypher("MATCH (d)-[r]->(c) WHERE type(r) <> 'IN' AND type(r) <> 'HAS_CHUNK' RETURN d,r,c")

GraphWidget(layout=Layout(height='800px', width='100%'))

## Time for Querying
We now demonstrate how to use both regular VectorRAG and GraphRAG to ask questions to the database:
- The VectorRAG implementation uses only the chunked text and their embeddings.
- The GraphRAG implementation also uses the context around the documents (extracted entities and their relationships) to generate an answer.

In [19]:
from neo4j import GraphDatabase
from neo4j_genai.retrievers import VectorRetriever, VectorCypherRetriever
from neo4j_genai.llm import OpenAILLM
from neo4j_genai.generation import GraphRAG
from neo4j_genai.embeddings.openai import OpenAIEmbeddings

index_name = "vecindex"
### Set up the RAG framework

# 1. Connect to Neo4j database
driver = GraphDatabase.driver(uri, auth=(username, password), database=database)

# 2. Create Embedder object, needed to convert the user question (text) to a vector
embedder = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. LLM
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

In [20]:
# Vector RAG:
retriever = VectorRetriever(driver, index_name, embedder)
# Initialize the RAG pipeline (note: this is just using plain vector RAG without context)
rag = GraphRAG(retriever=retriever, llm=llm)

# Query the graph
query_text = "Which Microsoft service can Reps on the road use?"
response = rag.search(query_text, retriever_config={"top_k": 3})
print("Response with VectorRAG:")
print(response.answer)

Response with VectorRAG:
Reps on the road can use bots to drive technical sales answers and get responses.


In [21]:
# Now, context-aware GraphRAG
retriever = VectorCypherRetriever(
    driver,
    index_name=index_name,
    retrieval_query="""
    // Retrieve Neighbourhood (Context)
    MATCH path=(node)<-[r:IN]-(node2)
    OPTIONAL MATCH (node2)-[r2]-(node3) 
    WHERE type(r2) <> "IN"
    RETURN node.index as chunkindex, node2.id as sourceid, node2.name as source, 
    toString(r2.id) as relid, type(r2) as reltype, 
    node3.id as targetid, node3.name as targetname, 
    score""",
    embedder=embedder
)

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retriever, llm=llm)

# Query the graph
response = rag.search(query_text, retriever_config={"top_k": 3}, return_context=True)
print("Response with GraphRAG:")
print(response.answer)

Response with GraphRAG:
Reps on the road can use the "Technical Sales Answers" service provided by Microsoft.


In [22]:
import re
# Define the regex pattern to match key-value pairs
pattern = re.compile(r"(\w+\.\w+|type\(r2\)|\w+)=('[^']*'|\d+\.\d+|\d+|[^' \n]+)")


used_context = [item.content for item in response.retriever_result.items]
used_context = [re.findall( pattern, item) for item in used_context]
used_context = [{key: value.replace("'","") for key, value in matches} for matches in used_context]

In [23]:
# Visualize the graph in Neo4j
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
widget = Neo4jGraphWidget(driver, overview_enabled=False, context_start_with=None)
widget.show_cypher("""
UNWIND $context as row
MATCH (c:Chunk)-[r1]-(n)-[r2]-(m) 
WHERE c.index = toInteger(row['chunkindex']) AND n.id = row['sourceid'] AND r2.id = toInteger(row['relid']) AND m.id = row['targetid']
RETURN c, r1, n, r2, m

""", context=used_context)

GraphWidget(layout=Layout(height='800px', width='100%'))